In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# Define a CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Data preprocessing and augmentation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download CIFAR-10 dataset and create data loaders
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=256, shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=256, shuffle=False)

# Initialize the model, loss function, and optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total:.2f}%')


100%|██████████| 170498071/170498071 [00:02<00:00, 84985738.61it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1, Batch 100, Loss: 1.748
Epoch 2, Batch 100, Loss: 1.317
Epoch 3, Batch 100, Loss: 1.171
Epoch 4, Batch 100, Loss: 1.078
Epoch 5, Batch 100, Loss: 1.008
Finished Training
Accuracy on the test set: 63.90%


Código da Aula


In [2]:
#@title Script that implements a convolutional neural network with PyTorch over the mnist 8 by 8 practice data set
# code adapted from https://github.com/rasbt/machine-learning-book/blob/main/ch14/ch14_part1.py

'''
This code does the following:
    Splits the dataset into training and testing sets.
    Standardizes the features using StandardScaler.
    Reshapes dataset to fit the model
    Instantiates the model (CNN)
    Defines the loss function (Cross Entropy Loss) and optimizer (Adam).
    Trains the model for num_epochs epochs.
    Tests the trained model on the test set and evaluates the accuracy.
'''

import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import  load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import random
import numpy as np

################################################################################ Data and parameters
SHOW=False # plot some digit for mnist 8*8

examples = load_digits() # https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html; 10 digits;  1797 examples
if SHOW:
    idx=random.randint(0,len(examples.target))
    print(examples.data[idx])
    print(examples.data[idx].reshape(8,8))
    print(examples.target[idx])
    plt.matshow(examples.data[idx].reshape(8,8), cmap=plt.cm.gray_r)
    plt.show()

X = examples.data # np.ndarray (1797, 64)
y = examples.target # (1797,)

# parameter constants
test_size=0.2
hidden_size = 8
batch_size= 256
num_epochs = 50
# Optimizer specific options
learning_rate=0.1
regularization_param=0.001
# Dropout: if p>0
dropout_p=0.1 # During training, randomly zeroes some of the elements of the input tensor with probability p.

########################################################################### train and test, pre-processing
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

# Standardize features
scaler = StandardScaler()
print(X_train.shape)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# mnist data set has examples with 64 attributes
# We need to reshape that information into NCHW (batch size, channels, height, width)
def reshape_mnist(X,W,H):
    X=X.reshape((X.shape[0],W,H))
    return np.expand_dims(X,1) # one channel

# Convert numpy arrays to PyTorch tensors of the right shape (labels do not need to be reshaped)
X_train_tensor = torch.tensor(reshape_mnist(X_train,8,8), dtype=torch.float32)
X_test_tensor = torch.tensor(reshape_mnist(X_test,8,8), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
print('Number of examples in training set:',X_train_tensor.shape)
print('Number of examples in test set:', X_test_tensor.shape)

# Instantiate the model
input_size = X_train_tensor.shape[1]
output_size = len(examples.target_names)

# Create dataloader and determine batch size (note: batchsize is the first parameter in NCHW)
train_dl=DataLoader(TensorDataset(X_train_tensor,y_train_tensor), batch_size, shuffle=True)
test_dl=DataLoader(TensorDataset(X_test_tensor,y_test_tensor), batch_size, shuffle=True)

if SHOW:
    class_names = [str(i) for i in range(10)]
    # Plot the images
    plt.figure(figsize=(10, 5))
    image_count = 0
    for images, labels in train_dl:
        for i in range(len(images)):
            plt.subplot(4, 5, image_count + 1)
            plt.imshow(np.transpose(images[i], (1, 2, 0)), cmap="gray")
            plt.title(class_names[labels[i]])
            plt.axis('off')
            image_count += 1
            if image_count >= 20:
                break
        if image_count >= 20:
            break
    plt.show()

###################################################################################### CNN  model
model=nn.Sequential(
    nn.Conv2d(in_channels=1,out_channels=8,kernel_size=3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(8*4*4, hidden_size),
    nn.BatchNorm1d(hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.BatchNorm1d(hidden_size),
    nn.ReLU(),
    nn.Dropout(p=dropout_p),
    nn.Linear(hidden_size, output_size)
)

'''
Compare with NN from previous script:
model=nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.BatchNorm1d(hidden_size),
    nn.ReLU(),
    nn.Dropout(p=dropout_p),
    nn.Linear(hidden_size, hidden_size),
    nn.BatchNorm1d(hidden_size),
    nn.ReLU(),
    nn.Dropout(p=dropout_p),
    nn.Linear(hidden_size, output_size)
)
'''
# model description
summary(model,(1,8,8)) # C, H, W

# Define loss function and optimizer
# Either torch.nn.NLLLoss or torch.nn.CrossEntropyLoss can be used: CrossEntropyLoss (https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) implements softmax internally
loss_fn = nn.CrossEntropyLoss()

# Optimizer: optimizer object that will hold the current state and will update the parameters based on the computed gradients
# for param in model.parameters(): print(param.data)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=regularization_param)

# Train the model and predict on test samples to estimate accuracy
# history stores losses, accuracy, actual labels and predictions
history = train(model, optimizer, loss_fn, num_epochs, train_dl, test_dl)

# plot losses along epochs
plot_losses(history)
# plot confusion matrix
plot_accuracy_from_predictions(history)
#plot_accuracy(hist)

(1437, 64)
Number of examples in training set: torch.Size([1437, 1, 8, 8])
Number of examples in test set: torch.Size([360, 1, 8, 8])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1              [-1, 8, 8, 8]              80
              ReLU-2              [-1, 8, 8, 8]               0
         MaxPool2d-3              [-1, 8, 4, 4]               0
           Flatten-4                  [-1, 128]               0
            Linear-5                    [-1, 8]           1,032
       BatchNorm1d-6                    [-1, 8]              16
              ReLU-7                    [-1, 8]               0
            Linear-8                    [-1, 8]              72
       BatchNorm1d-9                    [-1, 8]              16
             ReLU-10                    [-1, 8]               0
          Dropout-11                    [-1, 8]               0
           Linear-12             

NameError: name 'train' is not defined